In [ ]:
from __future__ import print_function
import torch.utils.data as data
from PIL import Image
import os
import os.path
import errno
import torch
from torch import nn
from torch.autograd import Variable
import json
import codecs
import numpy as np
import progressbar
import sys
import torchvision.transforms as transforms
import utils
import argparse
import json
from numpy.linalg import inv
import pickle
from datasets import Places365Dataset, ViewDataSet3D

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print('test')
parser = argparse.ArgumentParser()
parser.add_argument('--debug'  , action='store_true', help='debug mode')
parser.add_argument('--dataroot', required=True, help='path to dataset')
opt = parser.parse_args(['--dataroot', '/cvgl2/u/feixia/view_highres'])
tf = transforms.Compose([
    transforms.Scale(1024, 1024 * 2),
    transforms.ToTensor(),
])
    
mist_tf = transforms.Compose([
    transforms.ToTensor(),
])

d = ViewDataSet3D(root=opt.dataroot, debug=opt.debug, transform=tf, mist_transform=mist_tf, seqlen = 2, off_3d = False, off_pc_render = True)
print(len(d))
means = []


test
Processing the data:
view_highres_fofn1.pkl


In [ ]:
print(len(d))

In [ ]:
class Depth3DGridGen(nn.Module):
    def __init__(self, height, width):
        super(Depth3DGridGen, self).__init__()
        self.height, self.width = height, width
        self.grid = np.zeros( [self.height, self.width, 3], dtype=np.float32)
        self.grid[:,:,0] = np.expand_dims(np.repeat(np.expand_dims(np.arange(-1, 1, 2.0/self.height), 0), repeats = self.width, axis = 0).T, 0)
        self.grid[:,:,1] = np.expand_dims(np.repeat(np.expand_dims(np.arange(-1, 1, 2.0/self.width), 0), repeats = self.height, axis = 0), 0)
        self.grid[:,:,2] = np.ones([self.height, width])
        self.grid = torch.from_numpy(self.grid.astype(np.float32))

        self.theta = self.grid[:,:,0] * np.pi/2 + np.pi/2
        self.phi = self.grid[:,:,1] * np.pi

        self.x = torch.sin(self.theta) * torch.cos(self.phi)
        self.y = torch.sin(self.theta) * torch.sin(self.phi)
        self.z = torch.cos(self.theta)

        self.grid3d = torch.from_numpy(np.zeros( [self.height, self.width, 4], dtype=np.float32))

        self.grid3d[:,:,0] = self.x
        self.grid3d[:,:,1] = self.y
        self.grid3d[:,:,2] = self.z
        self.grid3d[:,:,3] = self.grid[:,:,2]


    def forward(self, depth, transformation):
        self.batchgrid3d = torch.zeros(torch.Size([depth.size(0)]) + self.grid3d.size())

        for i in range(depth.size(0)):
            self.batchgrid3d[i] = self.grid3d

        self.batchgrid3d = Variable(self.batchgrid3d)
        self.batchgrid = torch.zeros(torch.Size([depth.size(0)]) + self.grid.size())

        for i in range(depth.size(0)):
            self.batchgrid[i] = self.grid

        self.batchgrid = Variable(self.batchgrid)

        x = self.batchgrid3d[:,:,:,0:1] * depth * 128
        y = self.batchgrid3d[:,:,:,1:2] * depth * 128
        z = self.batchgrid3d[:,:,:,2:3] * depth * 128
        print(x.size(), y.size(), z.size())
        r = torch.sqrt(x**2 + y**2 + z**2)
        #print(r)
        theta = torch.acos(z/r)/(np.pi/2)  - 1
        phi = torch.atan(y/x)
        phi = phi/np.pi

        output = torch.cat([theta,phi], 3)
        
        return output

In [ ]:
gridgen = Depth3DGridGen(1024, 2048)

In [ ]:
data = d[0]

In [16]:
source = Variable(d[0][0]).cuda().unsqueeze(0)
target = Variable(d[1]).cuda().unsqueeze(0)
depth = Variable(d[2]).transpose(1,0).transpose(2,1)

[
 ( 0  ,.,.) = 
   0.6235  0.6235  0.6235  ...   0.6235  0.6235  0.6235
   0.6235  0.6235  0.6235  ...   0.6235  0.6235  0.6235
   0.6235  0.6235  0.6235  ...   0.6235  0.6235  0.6235
            ...             ⋱             ...          
   0.5098  0.5098  0.5098  ...   0.5059  0.5059  0.5059
   0.5098  0.5098  0.5098  ...   0.5059  0.5059  0.5059
   0.5098  0.5098  0.5098  ...   0.5059  0.5059  0.5059
 
 ( 1  ,.,.) = 
   0.6000  0.6000  0.6000  ...   0.6000  0.6000  0.6000
   0.6000  0.6000  0.6000  ...   0.6000  0.6000  0.6000
   0.6000  0.6000  0.6000  ...   0.6000  0.6000  0.6000
            ...             ⋱             ...          
   0.4471  0.4471  0.4471  ...   0.4510  0.4510  0.4510
   0.4471  0.4471  0.4471  ...   0.4510  0.4510  0.4510
   0.4471  0.4471  0.4471  ...   0.4510  0.4510  0.4510
 
 ( 2  ,.,.) = 
   0.6157  0.6157  0.6157  ...   0.6157  0.6157  0.6157
   0.6157  0.6157  0.6157  ...   0.6157  0.6157  0.6157
   0.6157  0.6157  0.6157  ...   0.6157  0.6157  0.61

In [ ]:
depth = Variable(d[2]).transpose(1,0).transpose(2,1)

In [ ]:
depth.size()

IndexError: tuple index out of range